In [18]:
import cv2
import numpy as np
from skimage.filters import sobel
def read_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img, gray

def compute_energy_map(gray_img):
    return sobel(gray_img)

def find_seam(energy_map):
    rows, cols = energy_map.shape
    seam = np.zeros(rows, dtype=np.int32)
 
    energy_table = np.copy(energy_map)
    for row in range(1, rows):
        for col in range(1, cols-1):
            min_energy = min(energy_table[row-1, col-1], energy_table[row-1, col], energy_table[row-1, col+1])
            energy_table[row, col] += min_energy
    
    seam[rows-1] = np.argmin(energy_table[rows-1])
    for row in range(rows-2, -1, -1):
        prev_col = seam[row+1]
        if prev_col == 0:
            seam[row] = np.argmin(energy_table[row, prev_col:prev_col+2])
        else:
            seam[row] = np.argmin(energy_table[row, prev_col-1:prev_col+2]) + prev_col-1
    
    return seam

def remove_seam(img, seam):
    rows, cols = img.shape[:2]
    for row in range(rows):
        col = seam[row]
        img[row, col:cols-1] = img[row, col+1:cols]
    return img[:, :cols-1]

# Đọc và xử lý 3 ảnh
image_paths = [
    "C:/Users/ngtrd/Desktop/Sandipan_Dey_2018_Sample_Images/images/messi.jpg",
    "C:/Users/ngtrd/Desktop/Sandipan_Dey_2018_Sample_Images/images/ronaldo.jpg",
    "C:/Users/ngtrd/Desktop/Sandipan_Dey_2018_Sample_Images/images/aero.jpg"
]

for path in image_paths:
    img, gray = read_image(path)
    
    energy_map = compute_energy_map(gray)
    
    seam = find_seam(energy_map)
    result = remove_seam(img, seam)
    
    cv2.imshow(f'Seam Carved Image - {path.split("/")[-1]}', result)
    cv2.waitKey(0)

cv2.destroyAllWindows()
